In [1]:
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from IPython.display import display 

import model_utils as utils

# Pretty display for notebooks
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

dataset, features, labels = utils.getDataSet("datasets/dataset_over_sampled.csv.gz")

,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,CIE_10,...,PQR_CLASE_SNS,PQR_ESTADO,PQR_TIPOATENCION,PQR_TIPOPETICION,TRIM,AFEC_LOCATION,ENT_LOCATION,PET_LOCATION,COMPLETE_MOTIVE,RIESGO_VIDA
0,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.445445,...,1.000000e-07,1.000000e-07,1.000000,1.000000e-07,1.000000e-07,0.246246,0.185686,0.053053,0.274775,0
1,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.445445,...,1.000000e-07,1.000000e-07,1.000000,1.000000e-07,6.481481e-01,0.076577,0.185686,0.368869,0.142142,0
2,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.445445,...,1.000000e-07,1.000000e-07,1.000000,1.000000e-07,1.000000e-07,0.076577,0.185686,0.890891,0.142142,0
3,5.805806e-01,5.970971e-01,5.270270e-01,7.267267e-01,9.454454e-01,7.082082e-01,9.999999e-01,9.999999e-01,1.000000e-07,0.239239,...,9.999999e-01,3.413413e-01,0.168669,9.919920e-01,3.806306e-01,0.246246,0.958458,0.845846,0.883383,0
4,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.453954,...,1.000000e-07,1.000000e-07,1.000000,1.000000e-07,9.999999e-01,0.076577,0.909910,0.618619,0.274775,0


## Shuffle and Split Data

In [2]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'labels' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size = 0.20, 
                                                    random_state = 10)

# Show the results of the split
print "features_final set has {} samples.".format(features.shape[0])
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

features_final set has 3315592 samples.
Training set has 2652473 samples.
Testing set has 663119 samples.


### Creating a Training and Predicting Pipeline

In [3]:
'''
TP = np.sum(income) # Counting the ones as this is the naive case. Note that 'income' is the 'income_raw' data 
encoded to numerical values done in the data preprocessing step.
FP = income.count() - TP # Specific to the naive case

TN = 0 # No predicted negatives in the naive case
FN = 0 # No predicted negatives in the naive case
'''

tp = float(np.sum(labels['RIESGO_VIDA']))
fp = float(labels['RIESGO_VIDA'].count() - tp)
tn = 0
fn = 0

# TODO: Calculate accuracy, precision and recall
recall = tp / (tp + fn)
precision = tp / (tp + fp)

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
# HINT: The formula above can be written as (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
beta = 2
fscore = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

# Print the results 
print "Naive Predictor: [F-score: {:.4f}]".format(fscore)

Naive Predictor: [F-score: 0.8333]


In [4]:
import visuals as vs
from sklearn.metrics import fbeta_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
# Initialize the three models
clf_A = SGDClassifier(random_state = 300)
clf_B = RandomForestClassifier(random_state = 300)
clf_C = AdaBoostClassifier(random_state = 300)


samples_100 = len(y_train)

# Collect results on the learners
dfResults = pd.DataFrame(columns=['learner', 'learner_index', 'size_index', 'train_time', 'pred_time', 'f_test', 'f_train'])

for k, clf in enumerate([clf_A, clf_B, clf_C]):
    clf_name = clf.__class__.__name__    
    dfResults = utils.train_predict(clf, k, 0, samples_100, X_train, y_train, X_test, y_test, dfResults)


/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier trained on 2652473 samples.


/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
model_utils.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  learner = learner.fit(X_train[:sample_size], y_train[:sample_size])


RandomForestClassifier trained on 2652473 samples.
AdaBoostClassifier trained on 2652473 samples.


In [5]:
display( dfResults.sort_values(by=['f_test'], ascending = False)[['learner', 'f_test']])

,learner,f_test
1,RandomForestClassifier,0.991604
2,AdaBoostClassifier,0.882436
0,SGDClassifier,0.882184


# Tuning Models


## Tuning SGDClassifier

In [7]:
sdgClassifier = SGDClassifier(random_state = 20)

sdgParameters = {
  'eta0':[0.01],
  'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
  'penalty':['none', 'l2', 'l1', 'elasticnet'],
  'learning_rate':['constant', 'optimal', 'invscaling'],
  'class_weight' :['balanced']
  #'max_features':['auto', 'sqrt', 'log2', None],
}

sdgClassifier = utils.tuneClassifier(sdgClassifier, sdgParameters, X_train, X_test, y_train, y_test)

joblib.dump(rfClassifier, 'sdgClassifier_over_sampled.joblib') 

Unoptimized model
------
F-score on testing data: 0.8910

Optimized Model
------
Final F-score on the testing data: 0.8983


['sdgClassifier_over_sampled.joblib']

## Testing Model with the validation dataset


In [14]:
dataset_excluded, features_excluded, labels_excluded = utils.getDataSet("datasets/dataset_over_sampled_validation.csv.gz")

from sklearn.base import clone
rfClassifier = (clone(clf_B)).fit(X_train, y_train)

predictions_excluded = sdgClassifier.predict(features_excluded)
f_test_excluded =  fbeta_score(labels_excluded, predictions_excluded, 2)
print(f_test_excluded)

,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,CIE_10,...,PQR_CLASE_SNS,PQR_ESTADO,PQR_TIPOATENCION,PQR_TIPOPETICION,TRIM,AFEC_LOCATION,ENT_LOCATION,PET_LOCATION,COMPLETE_MOTIVE,RIESGO_VIDA
0,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.445445,...,1.000000e-07,1.000000e-07,1.000000,1.000000e-07,1.000000e-07,0.076577,0.185686,0.053053,0.372873,0
1,5.805806e-01,5.970971e-01,5.270270e-01,7.267267e-01,5.580581e-01,7.082082e-01,5.730731e-01,9.999999e-01,1.000000e-07,0.445445,...,9.999999e-01,6.316316e-01,0.168669,6.571572e-01,4.119119e-01,0.779349,0.917076,0.437668,0.647648,0
2,9.999999e-01,5.970971e-01,9.999999e-01,7.267267e-01,9.114114e-01,7.082082e-01,8.268268e-01,9.999999e-01,1.000000e-07,0.377878,...,1.000000e-07,6.316316e-01,0.168669,1.000000e-07,1.000000e-07,0.865365,0.842843,0.890891,0.372873,0
3,7.374875e-01,9.096597e-01,5.270270e-01,7.267267e-01,9.454454e-01,9.789790e-01,5.730731e-01,9.999999e-01,1.000000e-07,0.445445,...,9.999999e-01,6.316316e-01,1.000000,8.298298e-01,1.000000e-07,0.703203,0.710711,0.514014,0.647648,0
4,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.445445,...,1.000000e-07,1.000000e-07,1.000000,1.000000e-07,9.999999e-01,0.246246,0.036036,0.262262,0.274775,0


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.6921063907933399
